**Introduction**

In order to understand a fair value of a stock price I will use several classic tools which are grouped by the term "Discounted Cash Flow model"  (see below). I named all variables by their full financial names and went step by step in order to make this notebook more educational and understandable.


**Discounted Cash Flow**

A discounted cash flow valuation is used to determine if an investment is worthwhile in the long run. For example, in investment banking, a DCF valuation is used to determine if a potential merger or acquisition is worth it. Additionally, DCF valuation is used in real estate and private equity. I will use this valuation for subjectevely measuring fair a stock price. The subjectivity was explained in the Assumptions section below. The script can be applied for any stocks in the US market.

$DCF = \frac {CF_1} {(1+r)} + \frac {CF_2} {(1+r)^2}+ \dots + \frac {CF_n} {(1+r)^n}$

Where 

$CF_n$ - Cash flow 

$r$ - Discount rate


**Capital Asset Pricing Model (CAMP)**

The goal of the CAPM formula is to evaluate whether a stock is fairly valued when its risk and the time value of money are compared with its expected return. In other words, by knowing the individual parts of the CAPM, it is possible to gauge whether the current price of a stock is consistent with its likely return. [1] The CAMP formula was used to evaluate Cost of Equity which, in turn, was used for calculations of Weighted average cost of capital (see below). 

$ER_i​=R_f​+β_i​(ER_m​−R_f​)$

where:

$ER_i$​ - expected return of investment;

$R_f$​ - risk-free rate;

$β_i$​ - beta of the investment;

$(ER_m​−R_f​)$ - market risk premium​.

**Weighted Average Cost of Capital (WACC)**

In corporate finance, determining a company’s cost of capital is vital for a couple of reasons. For instance, WACC is the discount rate that a company uses to estimate its net present value. I used this metric exactly for this reason for calculation NPV of Free cash flow (see below).

WACC is also important when analyzing the potential benefits of taking on projects or acquiring another business. For example, if the company believes that a merger will generate a return higher than its cost of capital, then it’s likely a good choice for the company. If its management anticipates a return lower than what their own investors are expecting, then they’ll want to put their capital to better use.[2]

In my work I calculated WACC as follows:

$WACC = R_{eq.}P_{eq.} + R_{debt}P_{debt}$

Where

$P_{eq.}$ - percentage of equity;

$P_{debt}$ - percentage of debt;

$R_{eq.}$ - Cost of equity;

$R_{debt}$ - Cost of debt.

Percentages were calculated with formulae:

$P_{eq.}$ = Total Equity / (Total liability + Total Equity)

$P_{debt}$ = 1 - $P_{eq.}$

**Net present value**

Net present value (NPV) is the difference between the present value of cash inflows and the present value of cash outflows over a period of time. NPV is used in capital budgeting and investment planning to analyze the profitability of a projected investment or project. I used NPV to discount free cash flow (FCF).

$NPV = \sum_{t=0}^{M-1} \frac {C_t}{(1+r)^t}$

Where

$C_t$ - Cash flow for a period $t$ - $t-1$;

$r$ - return or discount rate;

$t$ - time steps of cash flows.


**Free Cash Flow**

Free cash flow (FCF) is the cash a company generates after taking into consideration cash outflows that support its operations and maintain its capital assets. In other words, free cash flow is the cash left over after a company pays for its operating expenses and capital expenditures. [3] 

To calculate 

$FCF = EBIT * (1 - T) + D - C - W$

Where

$EBIT$ - Earnings before interest and taxes;

$T$ - Tax schield;

$D$ - Deprication & Amortization;

$C$ - Capital Expenditures;

$W$ - Change in Working Capital.


**Assumptions**

* This example is oriented to the US stock market. Therefore, I used S&P 500 index and the U.S. 10 Year Treasury Note to find Equity Risk Premium.
* I used simple returns for my calculations:

$Returns = (y_{t+1} - y_t) / y_t$

* Exponentially weighted moving average (EWMA) 
EWMA with a decay rate 0.5 was used to evaluate Equity Risk Premium $= R_a - R_f$ where $R_a$ - expected return on investment (median SP500 index EWMA annual returns); $R_f$ - risk free returns (rate) (median treasuries 10Y annual returns smoothened by EWMA). This choice was made to "smooth up" recent elevation of the US stock market to potential long term investors who want to see a bigger picture.

Formula for EWMA:

$y_t = \frac {x+t + (1 - \alpha)x_{t-1} + (1- \alpha)^2 x_{t-2}+...+(1- \alpha)^t x_0}{1+(1-\alpha)+(1-\alpha)^2+...+(1-\alpha)^t}$
where
$\omega_i = (1-\alpha)^i$ are weights for the moving average

* For some forecasts (Cash flow, Revenue, Expenses, EBIT and Tax rate) I made an assumption that the average growth of the last three year is constant

* For some forecasts (Depreciation, Capital Expenditures, Change in Working Capital) I made an assumption that these values should equal the average for the last four years.

* SP500 index average EV/EBITDA multiplier was used to evaluate Terminal value

* I used WACC as a discount rate for net present value (NPV):
$\sum_{t=0}^{M-1} \frac {C_t}{(1+WACC)^t}$
As you can see WACC used as a discount rate and $C$ are forecasted cash flows



Reference:
1. Kenton, Will. “What Is the Capital Asset Pricing Model (CAPM)?” Investopedia, Investopedia, 13 Dec. 2022, https://www.investopedia.com/terms/c/capm.asp. 
2. Sinra. “What Is the Weighted Average Cost of Capital (WACC)? Definition, Formula, and Example.” CFAJournal, 4 Sept. 2022, https://www.cfajournal.org/wacc/. 
3. Murphy, Chris B. “What Is the Formula for Calculating Free Cash Flow?” Investopedia, Investopedia, 10 Nov. 2022, https://www.investopedia.com/ask/answers/033015/what-formula-calculating-free-cash-flow.asp. 

**Technical notes**

This notebook is based on yfinance library. Sometimes financials is not scraped well and it brings bugs. 

Moreover, The following metric are not provided by yfinance library: 

* Average 'EV/EBITDA multiplier' for a company's industry. One can find it here: https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/vebitda.html

* 'Change in working capital' are not provided by the library. One can find it on yahoo finance by checking a stock of interest profile and following the path: Financials => Cash Flow => Operating Cash Flow => Change in working capital.

Please input these metrics into the model object manually.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf


#assistant functions

def get_financial(source, name):
    return source[source.index == name].T

def get_average_growth(financial_df):
    return np.mean(financial_df.iloc[::-1].pct_change())[0]

def forecast(financial_df,  Num_years):
    # average growth used for forecasting
    last_value = financial_df.iloc[0,-1]
    average_growth = np.mean(financial_df.iloc[::-1].pct_change())[0]
    Growth_const = (average_growth + 1)
    Schedule = [last_value * Growth_const**year for year in range(1,Num_years+1)]
    Schedule_list = [pd.Timestamp.now().year + year for year in range(1,Num_years+1)]
    Schedule = pd.DataFrame({'Schedule':Schedule,'Year':Schedule_list})
    return Schedule

def get_NPV(Cash_flow, rate):
    return sum([C_t/ (1+rate)**t for t,C_t in enumerate(Cash_flow)])



class DCF_model (object):

    def __init__(self,ticker,Change_in_working_capital,EV_EBITDA_multiplier):
        self.ticker = ticker 
        
        #download data from yahoo finance
        ticker_ = yf.Ticker(self.ticker)
        self.Financials = ticker_.financials
        self.Cash_flow = ticker_.get_cashflow(as_dict=False)
        self.Balance_sheet = ticker_.balance_sheet
        self.info = ticker_.info
        self.Change_in_working_capital = Change_in_working_capital
        self.EV_EBITDA_multiplier = EV_EBITDA_multiplier

        self.Equity_candles = yf.download(self.ticker, interval ='3mo', period='max')['Adj Close']
        self.SP500_candles = yf.download('ES=F', interval ='3mo', period='max')['Adj Close']
        self.Treasures = yf.download('^TNX', interval ='3mo', period='20y')['Adj Close']
    def __get_cost_of_equity(self):
        # get data from yahoo finance
        Equity_returns_annual = self.Equity_candles.resample('Y').ffill().pct_change()
        SP500_returns_annual = self.SP500_candles.resample('Y').ffill().pct_change()

        # get cost of equity
        R_a = SP500_returns_annual.ewm(com=0.5).mean().median() 
        R_f = self.Treasures.resample('Y').ffill().ewm(com=0.5).mean().median() / 100
        Equity_Risk_Premium = R_a - R_f

        if len(Equity_returns_annual) >  len(SP500_returns_annual):
            Covariance = np.cov(Equity_returns_annual[SP500_returns_annual.index],SP500_returns_annual)[0][0]
        else:
            Covariance = np.cov(Equity_returns_annual,SP500_returns_annual[Equity_returns_annual.index])[0][0]
        Variance = np.var(Equity_returns_annual)
        Beta = Covariance / Variance
        Cost_of_equity = Equity_Risk_Premium * Beta + R_f

        self.Cost_of_equity = Cost_of_equity
        self.Equity_Risk_Premium = Equity_Risk_Premium
        self.Beta = Beta
        self.Risk_Free_Rate = R_f

    def __get_cost_of_debt(self):
        # get financials from yahoo finance

        Total_liabilities = get_financial(self.Balance_sheet, 'Total Liab')
        Total_liabilities ['Interest Expense'] = get_financial(self.Financials, 'Interest Expense')
        Interest_payments_schedule = Total_liabilities['Interest Expense'] / Total_liabilities['Total Liab']

        Financials = get_financial(self.Financials, 'Income Before Tax').join(get_financial(self.Financials, 'Income Tax Expense'))
        Tax_schedule = Financials['Income Tax Expense'] / Financials['Income Before Tax']

        Average_yield_on_debt = np.mean(Interest_payments_schedule)  
        Tax_schield = np.mean(Tax_schedule)

        Cost_of_Debt = Average_yield_on_debt*(1 - Tax_schield)
        
        self.Cost_of_Debt = Cost_of_Debt
        self.Total_liabilities = Total_liabilities
        self.Tax_schedule = Tax_schedule
        self.Average_yield_on_debt = Average_yield_on_debt
        self.Tax_schield = Tax_schield

    def __get_WACC(self):
        percentage_of_equity =  get_financial(self.Balance_sheet, 'Total Stockholder Equity')
        percentage_of_equity['Total Liab'] =  get_financial(self.Balance_sheet, 'Total Liab')
        percentage_of_equity['percentage_of_equity'] = percentage_of_equity['Total Stockholder Equity'] / (percentage_of_equity['Total Liab'] + percentage_of_equity['Total Stockholder Equity'])
        percentage_of_equity = percentage_of_equity.iloc[0,-1] # last value

        percentage_of_debt = 1 - percentage_of_equity
        WACC = self.Cost_of_equity * percentage_of_equity + self.Cost_of_Debt*percentage_of_debt
        self.WACC = WACC

    def __get_DCF(self):
        Revenue = get_financial(self.Financials,'Total Revenue')
        Total_Operating_Expenses = get_financial(self.Financials,'Total Operating Expenses')
        EBIT = Revenue.iloc[:,0] - Total_Operating_Expenses.iloc[:,0]
        Revenue_growth_average = get_average_growth(Revenue)
        Expenses_growth_average = get_average_growth(Total_Operating_Expenses)

        Depreciation = get_financial(self.Cash_flow,'Depreciation')
        Capital_Expenditures = get_financial(self.Cash_flow,'Capital Expenditures') 

        Forecast_Revenue = forecast(Revenue,  5)
        Forecast_Operating_Expenses = forecast(Total_Operating_Expenses,  5)
        Forecast_Depreciation = pd.DataFrame({'Schedule': np.ones (5) * np.mean(Depreciation)[0], 'Year': Forecast_Revenue['Year']})
        Forecast_Capital_Expenditures = pd.DataFrame({'Schedule': np.ones (5) * np.mean(Capital_Expenditures)[0], 'Year': Forecast_Revenue['Year']})
        Forecast_Change_in_Working_Capital = pd.DataFrame({'Schedule': np.ones (5) * np.mean(self.Change_in_working_capital), 'Year': Forecast_Revenue['Year']})
        Forecast_Tax_rate = pd.DataFrame({'Schedule': np.ones (5) * self.Tax_schield, 'Year': Forecast_Revenue['Year']})

        Forecast_EBIT = pd.DataFrame()
        Forecast_EBIT ['Schedule']= Forecast_Revenue['Schedule'] - Forecast_Operating_Expenses['Schedule']
        Forecast_EBIT['Year'] = Forecast_Revenue['Year']

        
        EV_EBITDA_terminal_value = (Forecast_EBIT.iloc[-1,0] + Forecast_Depreciation.iloc[-1,0]) * self.EV_EBITDA_multiplier

        FCF = pd.DataFrame()
        FCF['Schedule'] = Forecast_EBIT['Schedule'] * (1 - Forecast_Tax_rate['Schedule']) + Forecast_Depreciation['Schedule'] - Forecast_Capital_Expenditures['Schedule'] - Forecast_Change_in_Working_Capital['Schedule']
        FCF['Year'] = Forecast_Revenue['Year']
        FCF_and_Terminal_value = list(FCF['Schedule']) + [EV_EBITDA_terminal_value]
        Enterprise_Value = get_NPV(FCF_and_Terminal_value, self.WACC)
        Cash_Equivalents = get_financial(self.Balance_sheet,'Cash')
        Debt = get_financial(self.Balance_sheet,'Short Long Term Debt')
        Equity_Value = Enterprise_Value + Cash_Equivalents.iloc[0,0] - Debt.iloc[0,0]


        Shares_outstanding = self.info['sharesOutstanding']
        Market_Price = self.info['regularMarketPrice']

        Fair_value_per_share = Equity_Value / Shares_outstanding
        Growth_forecast = (Fair_value_per_share / Market_Price) -1


        #final table
        final_dictionary_1 = {
         'Year': list(FCF['Year']) + ["Terminal"],
         'FCF and Terminal value': FCF_and_Terminal_value
        }
        final_list_headers = ['Enterprise value (NPV of FCF)', 'Fair Value per share','Market price','Growth forecast']
        final_list_values = [Enterprise_Value,Fair_value_per_share,Market_Price,Growth_forecast]
        final_list_values = [int(i) for i in final_list_values] 
        
        display (pd.DataFrame(final_dictionary_1))
        display (pd.DataFrame.from_dict({'headers':final_list_headers,'values':final_list_values}, orient = 'columns'))

    def results(self):
        self.__get_cost_of_equity()
        self.__get_cost_of_debt()
        self.__get_WACC()
        self.__get_DCF()


In [93]:
ticker = "AAPL"
Change_in_working_capital = [1200000, 1200000, -4911000, 5690000,-3488000]
EV_EBITDA_multiplier = 17.12

model = DCF_model(ticker,Change_in_working_capital,EV_EBITDA_multiplier)

model.results()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Year,FCF and Terminal value
0,2023,1.465743e+11
1,2024,1.748154e+11
2,2025,2.085116e+11
3,2026,2.486398e+11
4,2027,2.963446e+11
5,Terminal,5.732633e+12


,headers,values
0,Enterprise value (NPV of FCF),6682655243684
1,Fair Value per share,420
2,Market price,136
3,Growth forecast,2
